# First BERT Experiments

In this notebook we do some first experiments with BERT: we finetune a BERT model+classifier on each of our datasets separately and compute the accuracy of the resulting classifier on the test data.

For these experiments we use the `pytorch_transformers` package. It contains a variety of neural network architectures for transfer learning and pretrained models, including BERT and XLNET.

Two different BERT models are relevant for our experiments: 

- BERT-base-uncased: a relatively small BERT model that should already give reasonable results,
- BERT-large-uncased: a larger model for real state-of-the-art results.

In [1]:
BERT_MODEL = 'bert-base-uncased'
BATCH_SIZE = 16 if "base" in BERT_MODEL else 2
GRADIENT_ACCUMULATION_STEPS = 1 if "base" in BERT_MODEL else 8
MAX_SEQ_LENGTH = 100
PREFIX = "eatingmeat4_because"

## Data

We use the same data as for all our previous experiments. Here we load the training, development and test data for a particular prompt.

In [2]:
import sys
sys.path.append('../')

import ndjson
import glob
import numpy as np

from quillnlp.models.bert.preprocessing import preprocess, create_label_vocabulary

data_file = f"../data/interim/{PREFIX}_withprompt.ndjson"

with open(data_file) as i:
    data = ndjson.load(i)
        
# Make sure this is a single-label problem
label_lengths = [len(item["labels"]) for item in data]
assert max(label_lengths) == 1

for item in data:
    item["label"] = item["labels"][0]
        
label2idx = create_label_vocabulary(data)
idx2label = {v:k for k,v in label2idx.items()}
target_names = [idx2label[s] for s in range(len(idx2label))]

data_items = preprocess(data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH)
data_items = np.array(data_items)

I0404 09:56:08.656721 140621602613056 file_utils.py:41] PyTorch version 1.2.0+cu92 available.
I0404 09:56:09.640355 140621602613056 file_utils.py:57] TensorFlow version 2.1.0 available.
I0404 09:56:10.284965 140621602613056 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


## Training

In [3]:
import torch
import random

from quillnlp.models.bert.train import train, evaluate
from quillnlp.models.bert.models import get_bert_classifier

from quillnlp.models.bert.preprocessing import get_data_loader
from sklearn.model_selection import KFold

random.shuffle(data_items)

kf = KFold(n_splits=5, shuffle=True, random_state=1)
all_correct, all_predicted = [], []
all_test_data = []
for train_idx, test_idx in kf.split(data_items):

    train_and_dev_data = data_items[train_idx]
    cutoff = int(len(train_and_dev_data)/4*3)
    
    train_data = train_and_dev_data[:cutoff]
    dev_data = train_and_dev_data[cutoff:]
    test_data = data_items[test_idx]

    train_dataloader = get_data_loader(train_data, BATCH_SIZE)
    dev_dataloader = get_data_loader(dev_data, BATCH_SIZE)
    test_dataloader = get_data_loader(test_data, BATCH_SIZE, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = get_bert_classifier(BERT_MODEL, len(label2idx), device=device)
    output_model_file = train(model, train_dataloader, dev_dataloader, 
                              BATCH_SIZE, GRADIENT_ACCUMULATION_STEPS, device)
    
    print("Loading model from", output_model_file)
    device="cpu"

    model = get_bert_classifier(BERT_MODEL, len(label2idx), model_file=output_model_file, device=device)
    model.eval()
    
    _, _, test_correct, test_predicted = evaluate(model, test_dataloader, device)
    all_correct.extend(test_correct)
    all_predicted.extend(test_predicted)
    all_test_data.extend(test_data)


I0404 09:56:11.153807 140621602613056 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0404 09:56:11.154999 140621602613056 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd

../quillnlp/models/bert/train.py:146: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = softmax(logits.to('cpu'))



Loss history: []
Dev loss: 1.6724319458007812


Epoch:   5%|▌         | 1/20 [00:06<02:06,  6.65s/it]


Loss history: [1.6724319458007812]
Dev loss: 1.2261486517058477


...


Loss history: [1.573062194718255, 1.3266699181662664, 1.15270553694831, 0.9924946824709574, 0.8813459873199463, 0.8213360442055596, 0.8199739389949374, 0.7752813100814819, 0.7582661079035865, 0.7575248446729448, 0.7645473778247833, 0.7203055123488108, 0.7410715437597699, 0.733455134762658]
Dev loss: 0.7386853131983016



Epoch:  80%|████████  | 16/20 [01:43<00:25,  6.26s/it]


Loss history: [1.573062194718255, 1.3266699181662664, 1.15270553694831, 0.9924946824709574, 0.8813459873199463, 0.8213360442055596, 0.8199739389949374, 0.7752813100814819, 0.7582661079035865, 0.7575248446729448, 0.7645473778247833, 0.7203055123488108, 0.7410715437597699, 0.733455134762658, 0.7386853131983016]
Dev loss: 0.7507240325212479



Loss history: [1.573062194718255, 1.3266699181662664, 1.15270553694831, 0.9924946824709574, 0.8813459873199463, 0.8213360442055596, 0.8199739389949374, 0.7752813100814819, 0.7582661079035865, 0.7575248446729448, 0.7645473778247833, 0.7203055123488108, 0.7410715437597699, 0.733455134762658, 0.7386853131983016, 0.7507240325212479]
Dev loss: 0.744017438756095
No improvement on development set. Finish training.
Loading model from /tmp/model.bin


I0404 10:06:27.408982 140621602613056 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0404 10:06:27.410612 140621602613056 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd

## Evaluation

In [4]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

print("Test performance:", precision_recall_fscore_support(all_correct, all_predicted, average="micro"))
print(classification_report(all_correct, all_predicted, target_names=target_names))

Test performance: (0.8144475920679887, 0.8144475920679887, 0.8144475920679887, None)
              precision    recall  f1-score   support

  Feedback_1       0.91      0.96      0.93       282
 Feedback_10       0.00      0.00      0.00        15
  Feedback_2       0.84      0.95      0.89        22
  Feedback_3       0.67      0.15      0.25        13
 Feedback _6       0.63      0.79      0.70        42
  Feedback_4       0.65      0.52      0.58        33
  Feedback_5       0.79      0.91      0.85       205
  Feedback_7       0.64      0.79      0.71        43
  Feedback_8       0.69      0.33      0.45        33
  Feedback_9       1.00      0.06      0.11        18

    accuracy                           0.81       706
   macro avg       0.68      0.55      0.55       706
weighted avg       0.80      0.81      0.79       706



/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
c = 0
for item, predicted, correct in zip(all_test_data, all_predicted, all_correct):
    assert item.label_ids == correct
    c += (item.label_ids == predicted)
    print("{}#{}#{}".format(item.text, idx2label[correct], idx2label[predicted]))
    
print()
print(c, "/", len(all_test_data), "=", c/len(all_test_data))

Methane from cow burps harms the environment because cows burp about 30-50 gallons of methane gas into the atmosphere each day.#Feedback_5#Feedback_5
Methane from cow burps harms the environment because it is a greenhouse gas which contributes to global warming#Feedback_1#Feedback_1
Methane from cow burps harms the environment because the gas increases earth temperature.#Feedback_1#Feedback_1
Methane from cow burps harms the environment because it adds carbon to the atmospehre#Feedback_10#Feedback_8
Methane from cow burps harms the environment because it adds to the global warming effect.#Feedback_1#Feedback_1
Methane from cow burps harms the environment because it produces 14.5% of greenhouse gases worldwide.#Feedback_7#Feedback_7
Methane from cow burps harms the environment because they're burps contain methane an overall create 30 to 50 gallons of methane released into the atmosphere.#Feedback_5#Feedback_5
Methane from cow burps harms the environment because it burps out 30-50 gallo